# find all of Katie's data

In [1]:
import os
# single reps 
clips = [f for f in os.listdir('/home/wjin/projects/CLIP_seq/Kris_Katie/Data') 
         if 'eCLIP' in f and not f.endswith('.yaml')]

In [2]:
clips

['DLST-B_eCLIP',
 'DARS-B_eCLIP',
 'IgGH-B_eCLIP',
 'eCLIP_QC',
 'ERH-B_eCLIP',
 'MARS-A_eCLIP',
 'DLST-A_eCLIP',
 'ERH-A_eCLIP',
 'RPS5-B_eCLIP',
 'DLD-A_eCLIP',
 'RPS5-A_eCLIP',
 'KARS-B_eCLIP',
 'KARS-A_eCLIP',
 'DARS-A_eCLIP',
 'AIMP1-B_eCLIP',
 'NUP35-B_eCLIP',
 'MARS-B_eCLIP',
 'NUP35-A_eCLIP',
 'DLD-B_eCLIP',
 'AIMP1-A_eCLIP',
 'PCF11-A_eCLIP',
 'IgGH-A_eCLIP',
 'PCF11-B_eCLIP']

In [3]:
idrs = [f for f in os.listdir('/home/wjin/projects/CLIP_seq/Kris_Katie/Data') 
         if f.startswith('full_IDR_pipeline') and not f.endswith('yaml')]

In [4]:
idrs

['full_IDR_pipeline_2inputs_example11',
 'full_IDR_pipeline_2inputs_example21',
 'full_IDR_pipeline_2inputs_example33',
 'full_IDR_pipeline_2inputs_example23',
 'full_IDR_pipeline_2inputs_example22']

In [5]:
# get full path for idr peaks
allidrs = []
indir='/home/wjin/projects/CLIP_seq/Kris_Katie/Data'
for folder in idrs:
    prefix=os.path.join(indir, folder, 'results')
    allidr = [os.path.join(prefix,f) for f in os.listdir(prefix) if 'vs' in f and f.endswith('.bed')]
    allidrs += allidr

In [6]:
allidrs

['/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example11/results/DARS_A.vs.DARS_B.bed',
 '/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example11/results/MARS_A.vs.MARS_B.bed',
 '/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example11/results/AIMP1_A.vs.AIMP1_B.bed',
 '/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example11/results/ERH_A.vs.ERH_B.bed',
 '/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example33/results/IgGH_A.vs.IgGH_B.bed',
 '/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example33/results/KARS_A.vs.KARS_B.bed']

In [7]:
# this is an internal package I wrote for easy access to encode data 
from pybedtools import BedTool
import matplotlib.pyplot as plt
plt.style.use('seaborn-muted')
import sys
import pandas as pd
import numpy as np
sys.path.append('/home/hsher/Metadensity/scripts')
from dataloader import encode_data, encode4_data
import pandas as pd


## Regional distribution for rG4

In [8]:

def get_jaccard(bed1,bed2):
    '''
    input 2 bed files, return jaccard index to represent how they overlap
    
    '''
    # select IDRs and G4 in UTR only
    
    
    
    intersect = len(bed1.intersect(bed2, s = True, u = True))
    not_one = len(bed2)-intersect
    not_two = len(bed1)-intersect
    
    jaccard = intersect/(intersect+not_one+not_two)
    return jaccard

In [9]:
# get a list of all encode IDR files

In [10]:
master_df = pd.concat([encode_data, encode4_data])

In [11]:
master_df['idr'] # these are all the IDR peaks!

0      /projects/ps-yeolab5/encode/EVN_eCLIP_analysis...
1      /projects/ps-yeolab5/encode/EVN_eCLIP_analysis...
2      /projects/ps-yeolab5/encode/EVN_eCLIP_analysis...
3      /projects/ps-yeolab5/encode/EVN_eCLIP_analysis...
4      /projects/ps-yeolab5/encode/EVN_eCLIP_analysis...
                             ...                        
437    /projects/ps-yeolab5/encore/processing/encore_...
441    /projects/ps-yeolab5/encore/processing/encore_...
445    /projects/ps-yeolab5/encore/processing/encore_...
449    /projects/ps-yeolab5/encore/processing/encore_...
453    /projects/ps-yeolab5/encore/processing/encore_...
Name: idr, Length: 335, dtype: object

In [12]:
# let's try out 1 bed files
katie_dars = BedTool('/home/wjin/projects/CLIP_seq/Kris_Katie/Data/full_IDR_pipeline_2inputs_example11/results/DARS_A.vs.DARS_B.bed')
encode_rbfox2 = BedTool(master_df.loc[master_df['RBP']=='GARS', 'idr'].iloc[0])

In [13]:
get_jaccard(katie_dars,encode_rbfox2)

0.09447852760736196

In [14]:
# let's loop over all encode proteins
jaccard_to_dars = {}
for index, row in master_df.iterrows():
    try:
        encode_rbp = row['uid']+'_'+ row['RBP'] # make a unique id for each RBP
        encode_bed = BedTool(row['idr'])
    
        jaccard_to_dars[encode_rbp] = get_jaccard(katie_dars, encode_bed)
    except Exception as e:
        print(e) # some of the encode data are not complete yet therefore it might elicit some error

File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master_batch11a_IDR/results/4038_CLIP1_rep1.vs.4038_CLIP2_rep2.bed" does not exist
File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master_batch11a_IDR/results/4053_CLIP1_rep1.vs.4053_CLIP2_rep2.bed" does not exist
File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master_batch14_IDR/results/4099_CLIP1_rep1.vs.4099_CLIP2_rep2.bed" does not exist
File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master_batch16_IDR/results/4093_CLIP1_rep1.vs.4093_CLIP2_rep2.bed" does not exist
File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master__IDR/results/4094_CLIP1_rep1.vs.4094_CLIP2_rep2.bed" does not exist
File "/projects/ps-yeolab5/encore/processing/encore_master_IDR_hg38/results/encore_master_batch17_IDR/results/4096_CLIP1_rep1.vs.4096_CLIP2_rep2.bed" does not exist
File "/projects

In [15]:
#let's take a look at the output
jaccard_to_dars['204_RBFOX2']

0.0

In [16]:
jaccard_to_dars['203_HNRNPC']

0.0

In [17]:
# let's look at which RBP intersect with it the most
s = pd.Series(jaccard_to_dars) # convert to Series for easier manipulation
s.sort_values(ascending = False) # hoe does this look?

4059_DAP5      0.199301
4058_SRSF5     0.155556
4190_CACTIN    0.148499
4055_FUBP1     0.098947
4040_GARS      0.094479
                 ...   
684_ZC3H11A    0.000000
388_TAF15      0.000000
683_ZC3H8      0.000000
405_NONO       0.000000
203_HNRNPC     0.000000
Length: 328, dtype: float64